<a href="https://colab.research.google.com/github/gihuni99/Basic_MLDL/blob/main/Ch03_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-NN(Nearest Neighbors) Regression

supervised learning은 크게 classification과 regression으로 나뉜다.
* Classification: Sample을 몇개의 Class 중 하나로 분류
* Regression: Class 중 하나로 예측하는 것이 아닌, 임의의 수치를 예측(정해진 Class x)

**Regression(회귀)**: 두 변수 사이의 상관관계를 분석하는 방법

K-NN Classification은 새로운 Sample에 대해서 가장 가까운 이웃의 Class가 무엇인지 확인하고, sample의 class를 예측

K-NN Regression에서는 가장 가까운 이웃의 수치를 확인하고, 이를 평균하여 계산

농어의 무게를 예측하는 문제

### Data 준비

In [ ]:
import numpy as np

파이썬 배열을 numpy 배열로 변환하지 않고, 바로 numpy 배열로 선언

In [ ]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0,
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5,
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5,
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0,
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0,
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0,
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0,
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0,
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0,
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0,
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0,
     1000.0, 1000.0]
     )

아래 결과를 통해 농어의 길이가 늘어남에 따라, 무게가 증가하는 것을 알 수 있다.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(perch_length,perch_weight)
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

농어의 데이터를 train set과 test set으로 나눈다.

length에 따라 weight를 예측해야 하기 때문에 input data는 length, target data는 weigth가 된다.

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target=train_test_split(perch_length,perch_weight,random_state=42)


Scikit-learn에서는 train set가 2차원 배열이어야 한다.

따라서 [1,2,3]의 형태인 (3,)의 1차원 배열을, [[1],[2],[3]]의 형태인 (3,1)의 2차원 배열로 바꾸어야 한다.

numpy는 배열의 크기를 바꿀 수 있는 **reshape()** method를 제공한다

In [ ]:
#Ex) (4,) array -> (2,2) array
test_array=np.array([1,2,3,4])
print(test_array.shape)

**reshape()** method를 통해 배열의 크기를 바꾼다. reshape() method에 바꾸고 싶은 배열의 크기를 입력하면 된다.

하지만 (4,) 배열을 (2,3) 배열로 변환하려고 시도한다면 error가 발생한다.(element의 개수가 다르기 때문)

In [ ]:
test_array=test_array.reshape(2,2)
print(test_array.shape)

reshap() method를 사용하여 데이터의 차원을 2차원으로 변환해보자

In [ ]:
train_input=train_input.reshape(42,1)
test_input=test_input.reshape(14,1)
print(train_input.shape,test_input.shape)

위처럼 train data는 42개, test data는 14개이므로 reshape을 통해 변환 가능하다.

추가적으로 reshape() method는 배열의 크기를 자동으로 지정하는 기능도 있다.

예를 들어 train_input.reshape(-1,1)을 사용하면 자동으로 (42,1)의 배열로 지정된다.

만약 train_input.reshape(-1,2)를 하면 (21,2)의 배열로 변환될 것이다.

In [ ]:
train_input=train_input.reshape(-1,1)
test_input=test_input.reshape(-1,1)
print(train_input.shape,test_input.shape)

### 결정계수 $ R^2 $ (coefficient of determination)

Scikit-Learn에서 K-NN Regression을 구현한 클래스는 **KNeighborsRegressor**이다.(KNeighborsClassifier와 유사)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knr=KNeighborsRegressor()

knr.fit(train_input,train_target) #fit()함수를 통해 모델 학습

아래 결과를 보면 높은 점수가 나온 것을 볼 수 있다. 하지만 Regression에서는 Classification과 score방식이 다르다.

Classification에서는 test set에서 Sample의 Class를 얼마나 맞추었는지, 즉 정답의 비율을 Accuracy로 놓았다.

하지만 Regression에서는 당연하게도 정확한 수치를 예측할 수 없다. 따라서 결정계수($ R^2 $)를 통해서 모델을 평가한다.

$R^2$의 계산 방식은 다음과 같다.

$R^2$=$(타깃(정답값)-예측)^2의 합/(타깃(정답값)-타깃의 평균)^2의 합$

즉, 타깃의 평균을 예측하는 수준이라면(분자와 분모가 비슷해짐) $R^2$는 0에 가까워지고, 예측이 타깃에 아주 가까워지면(분자가 0에 가까워짐) 1에 가까운 값이 된다.

+) score() method가 출력하는 값은 높을수록 좋은 것

In [ ]:
print(knr.score(test_input,test_target))

scikit-learn에서는 여러가지 오차측정 tool을 제공한다.(**sklearn.metrics**패키지)

* **mean_absolute_error**: target(정답)과 예측의 절대값 오차를 평균하여 반환

아래 결과를 보면 결과와 예측이 평균적으로 19만큼 다르다는 것을 볼 수 있다.

In [ ]:
from sklearn.metrics import mean_absolute_error

test_prediction=knr.predict(test_input) #test data에 대한 예측치

mae=mean_absolute_error(test_target,test_prediction)
print(mae)

### Overfitting vs Underfitting

앞서 훈련한 모델을 사용하여 train set의 $R^2$점수를 확인

In [ ]:
print(knr.score(train_input,train_target))

위 결과를 보면 train set의 $R^2$가 test set의 $R^2$보다 낮은 것을 볼 수 있다.

* **Overfitting**:train set에서  score가 좋은데, test set에서 점수가 매우 낮다면 model이 overfitting되었다고 한다.(즉 training set에만 잘 맞는 모델)
* **Underfitting**:train set보다 test set에서 점수가 높거나, 두 점수 모두 너무 낮은 경우(당연히 train set에서의 점수가 더 높게 나와야 하는 것이 맞다. 하지만 model이 너무 단순하여 train set에 적절하게 학습되지 않거나, data의 양이 충분하지 않을 때 발생한다)

따라서 위의 경우는 **Underfitting**되었다고 볼 수 있다.

일반적으로 모델이 너무 단순할 때 Underfitting이 자주 발생한다. 따라서 모델의 복잡도를 높여보자.

K-NN Algorithm에서 복잡도를 높이는 방법은 K(neighbors)의 수를 줄이는 것이다. 따라서 K=5 -> K=3으로 설정

In [ ]:
knr.n_neighbors=3

knr.fit(train_input,train_target) #모델 다시 학습
print(knr.score(train_input,train_target)) #train set에 대한 모델 평가

In [ ]:
print(knr.score(test_input,test_target))

위 결과를 통해 train set에 대한 정확도를 높였고, test set의 정확도는 train set보다 낮은 것을 볼 수 있다.

**Underfitting 해결**

### 추가 공부

**Scikit-Learn**
* **KNeighborsRegressor**:K-NN Regression 모델을 만드는 scikit-learn 클래스이다. n_neighbors parameter로 K의 개수 지정(기본값은 5). 다른 parameter는 KNeighborsClassifier와 거의 동일
* **mean_absolute_error()**: Regression 모델의 평균 절대값 오차(Mean Absolute Error)를 계산한다. 첫번째 parameter에 target(정답), 두번째 parameter에 예측값 전달.
* **mean_squared_error()**: 평균 제곱 오차(Mean Squared Error)를 계산


**Numpy**
* reshape(): 배열의 크기를 바꾸는 method. **test_array.reshape(2,2)** 또는 **np.reshape(test_array,(2,2))** 두 형태 모두 사용 가능